# Load data

In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv("data/train.csv", encoding="ISO-8859-1")
test_df = pd.read_csv("data/test.csv", encoding="ISO-8859-1")
attribute_df = pd.read_csv("data/attributes.csv")
product_df = pd.read_csv("data/product_descriptions.csv")

In [3]:
# dataset = pd.concat((df_train, df_test), axis=0, ignore_index=True)
dataset = train_df

In [4]:
dataset = pd.merge(dataset, product_df, how='left', on='product_uid')

# Attributes processing

Exploration du dataframe attribute_df

In [5]:
attribute_df[0:3]

,product_uid,name,value
0,100001,Bullet01,Versatile connector for various 90° connection...
1,100001,Bullet02,Stronger than angled nailing or screw fastenin...
2,100001,Bullet03,Help ensure joints are consistently straight a...


In [6]:
print "unique product_uid in attribute_df: " + str(len(attribute_df["product_uid"].unique()))
print "unique attributes:  " + str(len(attribute_df["name"].unique()))

unique product_uid in attribute_df: 86264
unique attributes:  5411


In [7]:
print type(attribute_df.iloc[7]["value"])
attribute_df.iloc[7]["value"]

<type 'str'>


'12'

## 1) Bullets

### Identify bullet attributes
On veut concatener les données textes de description & bullets  
... à moins que les lecteurs regardent les bullets points en priorité ?

In [8]:
import re
myregex = r'^Bullet\d{1,2}'
attributes = attribute_df["name"].unique().tolist()
bullet_cols = [attr for attr in attributes if (re.match(myregex, unicode(attr)))]

## 2) Attributes that can appear multiple times for 1 product

### Understand the issue
Some attributes appear multiple times so we need to deal with them ...  
for example a product can be available in different colors, sizes, ...

In [9]:
investiguation = attribute_df.groupby(["name", "product_uid"]).count()
multi_attr = investiguation[investiguation["value"] > 1]

In [10]:
multiple_attr = set([el[0] for el in investiguation[investiguation["value"] > 1].index.values])
print str(len(multiple_attr)) + " different attributes may have multiple values"
print "5 examples: " + str(list(multiple_attr)[0:5])

45 different attributes may have multiple values
5 examples: ['Style', 'Seating Capacity', 'Power Source', 'Pool Type', 'Floor Options']


### Concatenate the values

In [11]:
new_attribute_df = attribute_df.dropna()
new_attribute_df = new_attribute_df.groupby(["name", "product_uid"])['value'].apply(lambda x: ' '.join(x))
new_attribute_df = new_attribute_df.reset_index()

In [12]:
print "len(attribute_df) : " + str(len(attribute_df))
print "len(new_attribute_df) : " + str(len(new_attribute_df))
print "number of lines aggregated : " + str(len(attribute_df) - len(new_attribute_df))

len(attribute_df) : 2044803
len(new_attribute_df) : 2039885
number of lines aggregated : 4918


## 3) Attributes to dataset

idée :  
si un terme de la query est proche d'un nom d'attribut,...  
... on veut voir si un autre terme de la query est égale à la valeur de l'attribut en question  
on veut donc faire :
- a) une jointure entre dataset et attributes_df
- b) un *cosine(query, name).cosine(query, value)*
- c) un pivot pour mettre ces features en colonne

### a)

In [13]:
dataset = pd.merge(dataset, new_attribute_df, how='left', on='product_uid')

In [14]:
print len(dataset)
print len(dataset["product_uid"].unique())
print len(dataset["name"].unique())

1425452
54667
4959


### b)

Some product don't have any attributes so we need to be careful when we train our tfidf to avoid error

In [22]:
print len(dataset["product_title"])-len(dataset["product_title"].dropna())
print len(dataset["search_term"])-len(dataset["search_term"].dropna())
print len(dataset["product_description"])-len(dataset["product_description"].dropna())
print len(dataset["name"])-len(dataset["name"].dropna())
print len(dataset["value"])-len(dataset["value"].dropna())

0
0
0
17520
17520


#### Train tfidf model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import itertools
mytfidf = TfidfVectorizer(strip_accents='unicode')
myiterable = itertools.chain(dataset["product_title"],
                             dataset["search_term"],
                             dataset["product_description"],
                             dataset["name"].dropna(),
                             dataset["value"].dropna())

mytfidf.fit(myiterable)

/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2645: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents='unicode', sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

#### Transform tfidf & compute cosine similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def f_title(row):
    title_vec = mytfidf.transform([row['product_title']])
    query_vec = mytfidf.transform([row['search_term']])
    sim = cosine_similarity(title_vec, query_vec)
    return float(sim)

def f_description(row):
    descr_vec = mytfidf.transform([row['product_description']])
    query_vec = mytfidf.transform([row['search_term']])
    sim = cosine_similarity(descr_vec, query_vec)
    return float(sim)

def f_attribute(row):
    attr_name_vec = mytfidf.transform([row['name']])
    attr_value_vec = mytfidf.transform([row['value']])
    query_vec = mytfidf.transform([row['search_term']])
    sim = cosine_similarity(attr_name_vec, query_vec) * cosine_similarity(attr_value_vec, query_vec)
    return float(sim)

dataset["f_title"] = dataset.apply(f_title, axis=1)
dataset["f_description"] = dataset.apply(f_description, axis=1)
dataset["f_attribute"] = dataset.apply(f_attribute, axis=1)

### c)

We add identifier for our attributes

In [ ]:
new_attribute_df["attribute_uid"] = pd.factorize(new_attribute_df.name)[0]
new_attribute_df["name_uid"] = new_attribute_df["attribute_uid"].apply(lambda x:"f_attribute"+unicode(x))
nb_f_attr = len(dataset["name_uid"])

We do the pivot to have our features in columns

In [ ]:
new_dataset = dataset.pivot(index=None,
                            columns='name_uid',
                            values='value')

In [ ]:
new_dataset[0:3]

# Train our ML model

We create our test set & our training set

In [ ]:
import numpy as np
seed = 42
ratio = 0.9
np.random.seed(seed)
msk = np.random.rand(len(new_dataset)) < ratio
training = new_dataset[msk]
test = new_dataset[~msk]

We train a RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
mymodel = RandomForestRegressor()
feature_list = ["f_title", "f_description"] + ["f_attribute"+str(i) for i in range(nb_f_attr)]
mymodel.fit(X=training[feature_list], y=training["relevance"])

# next steps  
- tester le modele actuel
- réduire le nombre de features (clustering noms d'attributs, réduction de dimension, ...)
- améliorer le text processing
- passer d'un cosine à un soft-tfidf

# TESTS

### we add ["attribute_uid", "name_uid", "value_uid"]

In [14]:
new_attribute_df["attribute_uid"] = pd.factorize(new_attribute_df.name)[0]
new_attribute_df["name_uid"] = new_attribute_df["attribute_uid"].apply(lambda x:"name"+unicode(x))
#attribute_df["value_uid"]= attribute_df["attribute_uid"].apply(lambda x:"value"+unicode(x))

In [15]:
new_attribute_df[0:3]

,name,product_uid,value,attribute_uid,name_uid
0,# of Line Wires,123462,1,0,name0
1,# of Line Wires,124031,1,0,name0
2,# of Line Wires,125205,1,0,name0


### we do the pivot

In [ ]:
nameid_df = new_attribute_df.pivot(index=None,
                                   columns='name_uid',
                                   values='value')
nameid_df

### we build an attribute dictionary

j'ai aussi besoin du dictionnaire name_attr[value_uid]  
pour que si any(token_query) in name_attr[value_uid]  
alors sim(query, valuei) sinon 0

In [77]:
tmp = attribute_df.groupby(["value_uid", "name"]).count().reset_index()[["value_uid", "name"]]
name_attr = pd.Series(tmp["name"].values, index=tmp["value_uid"]).to_dict()
name_attr
name_attr["value0"]

'Bullet01'

In [57]:
test = attribute_df[0:100].pivot(index='product_uid',
                                 columns='value_uid',
                                 values='value')
print test.iloc[1]["value27"]
print type(test.iloc[1]["value27"])

75
<type 'str'>


In [58]:
test

value_uid,value0,value1,value10,value11,value12,value13,value14,value15,value16,value17,...,value68,value69,value7,value70,value71,value72,value73,value74,value8,value9
product_uid,,,,,,,,,,,,,,,,,,,,,
100001,Versatile connector for various 90° connection...,Stronger than angled nailing or screw fastenin...,1,1.5,3,0.26,3,NaN,NaN,NaN,...,NaN,NaN,12,NaN,NaN,NaN,NaN,NaN,Galvanized Steel,Simpson Strong-Tie
100002,"Revives wood and composite decks, railings, po...",100% acrylic solid color coating,NaN,NaN,NaN,NaN,NaN,"Brush,Roller,Spray",6.63 in,7.76 in,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BEHR Premium Textured DeckOver
100003,Slightly narrower for tighter spaces,Designed with an 18 in. apron,NaN,NaN,NaN,129,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Composite,STERLING
100004,Positive power tolerance (0 to +5-Watt),Anti-reflective and anti-soiling surface reduc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,No,No,NaN,4,1.57,64.96,40,39,NaN,Grape Solar


In [10]:
attribute_df.count()

product_uid      2044648
name             2044648
value            2042713
attribute_uid    2044803
name_uid         2044803
value_uid        2044803
dtype: int64

Pour un product_uid, on ne doit avoir qu'un jeux de paramètres uniques

In [50]:
investiguation = attribute_df.groupby(["name", "product_uid"]).count()

In [51]:
investiguation.columns

Index([u'value'], dtype='object')

In [54]:
investiguation[investiguation["value"] > 1][0:3]

value
name           product_uid       
Accessory Type 103855           2
               128965           2
               137573           2

In [16]:
cond = (product_df["product_uid"] == 103855)
product_df[cond]["product_description"].values

array([ "Polar's cargo cover seals tight around your Polar Trailer bed, protecting gear from snow, rain and mud. This cover is water resistant. Designed to fit the LT 800 and Utility Cart.Protects gear and loadDesigned to fit the LT 800 and utility cartWater resistantCovers nicely for storage"], dtype=object)

contatener les champs "Accessory Type" et regarder quoi faire au mieux pour les autres

In [17]:
cond = (attribute_df["product_uid"] == 103855)
attribute_df[cond]

,product_uid,name,value,attribute_uid,name_uid,value_uid
91424,103855,Accessory Type,Other Accessory,865,name865,value865
91425,103855,Accessory Type,Travel Cover,865,name865,value865
91426,103855,Bullet01,Protects gear and load,0,name0,value0
91427,103855,Bullet02,Designed to fit the LT 800 and utility cart,1,name1,value1
91428,103855,Bullet03,Water resistant,2,name2,value2
91429,103855,Bullet04,Covers nicely for storage,3,name3,value3
91430,103855,Material,Fabric,8,name8,value8
91431,103855,MFG Brand Name,Polar Trailer,9,name9,value9
91432,103855,Product Depth (in.),52,11,name11,value11
91433,103855,Product Height (in.),6,12,name12,value12


In [18]:
cond = (attribute_df["name"] == "Accessory Type") & (attribute_df["product_uid"] == 103855)
attribute_df[cond]

,product_uid,name,value,attribute_uid,name_uid,value_uid
91424,103855,Accessory Type,Other Accessory,865,name865,value865
91425,103855,Accessory Type,Travel Cover,865,name865,value865


In [29]:
multiple_attr = set([el[0] for el in investiguation[investiguation["value"] > 1].index.values])
print len(multiple_attr)
multiple_attr

45


{'Accessory Type',
 'Alarm Sensor Type',
 'Application Method',
 'Collection Name',
 'Color Family',
 'Color/Finish Family',
 'Compressor Type',
 'Door Type',
 'Engine Make',
 'Engine Type',
 'Exhaust Vent Location',
 'Finish Family',
 'Floor Options',
 'Flooring Product Type',
 'Frame Material',
 'Garland Length (ft.)',
 'Gas Type',
 'Housing Material',
 'Indoor/Outdoor',
 'Inside Diameter',
 'Interior/Exterior',
 'Lock Type',
 'Material',
 'Mount Type',
 'Number of Faucet Handles',
 'Number of Pieces',
 'Part/Accessory Type',
 'Pool Type',
 'Power Source',
 'Power Type',
 'Product Type',
 'Pump Brand',
 'Pump Type',
 'Room Type',
 'Seating Capacity',
 'Shape',
 'Start Type',
 'Storage Product Type',
 'Style',
 'Switch Type',
 'Tape Length (ft.)',
 'Tool Type',
 'Tools Included',
 'Trip Lever Location',
 'Type'}

In [55]:
investiguation[investiguation["value"] > 1].count()

value            2824
attribute_uid    2824
name_uid         2824
value_uid        2824
dtype: int64

In [61]:
investiguation.count()

value            2041800
attribute_uid    2041800
name_uid         2041800
value_uid        2041800
dtype: int64

In [17]:
attribute_df[0:3].pivot(index=None,
                   columns='name_uid',
                   values='value')

name_uid,name0,name1,name2
0,Versatile connector for various 90° connection...,NaN,NaN
1,NaN,Stronger than angled nailing or screw fastenin...,NaN
2,NaN,NaN,Help ensure joints are consistently straight a...


In [ ]:
attribute_df[0:2].pivot(index=None,
                   columns='name_uid',
                   values='value')

In [44]:
help(attribute_df.pivot)

Help on method pivot in module pandas.core.frame:

pivot(self, index=None, columns=None, values=None) method of pandas.core.frame.DataFrame instance
    Reshape data (produce a "pivot" table) based on column values. Uses
    unique values from index / columns to form axes and return either
    DataFrame or Panel, depending on whether you request a single value
    column (DataFrame) or all columns (Panel)
    
    Parameters
    ----------
    index : string or object, optional
        Column name to use to make new frame's index. If None, uses
        existing index.
    columns : string or object
        Column name to use to make new frame's columns
    values : string or object, optional
        Column name to use for populating new frame's values
    
    Notes
    -----
    For finer-tuned control, see hierarchical indexing documentation along
    with the related stack/unstack methods
    
    Examples
    --------
    >>> df
        foo   bar  baz
    0   one   A    1.
    1   

In [5]:
dataset[:3]

,id,product_uid,product_title,search_term,relevance,product_description
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.0,"Not only do angles make joints stronger, they ..."
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.5,"Not only do angles make joints stronger, they ..."
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.0,BEHR Premium Textured DECKOVER is an innovativ...


je veux passer de...  
id | name | value  
5    name1  value1  
5    name2  value2

à...  
id | name | value | name | value  
id  name1  value1  name2  value2

In [37]:
attributes = attribute_df["name"].unique().tolist()
attributes[500:515]

['Container Width (in.)',
 'Planter Features',
 'Accessory type',
 'Stair Part Type',
 'Bullet Proof',
 'Mirrored',
 'Non-Glare',
 'Shatter Resistant',
 'Sheet Features',
 'Surface Type',
 'Dishwasher drain connection',
 'Disposer Feed',
 'Grind chamber capacity (oz.)',
 'Grinding Speed (RPM)',
 'Horsepower (hp)']